In [2]:
cd /content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN/CNN LSTM GRU

/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN/CNN LSTM GRU


# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import collections
import pickle
import gensim
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import re
import random
import sys
import os 
import pandas as pd
from sklearn.model_selection import train_test_split
from __future__ import print_function

from numpy import array
from numpy import asarray
from numpy import zeros
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.layers import Dropout, Activation, Flatten,Embedding, Convolution1D, MaxPooling1D, AveragePooling1D, Input, Dense, merge,Add
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.datasets import imdb
from keras import callbacks
from keras.utils import generic_utils
from keras.models import Model
from keras.optimizers import Adadelta
import time

Using TensorFlow backend.


# preprocess 1

In [0]:
def text_preprocessing(train_data,test_data):
  train_data_texts = train_data['comment']
  train_data_labels = train_data['label']
  test_data_texts = test_data['comment']
  test_data_labels = test_data['label']


  comment_texts = []
  comment_labels = []

  train_text = []
  test_text = []
  train_labels=[]
  test_labels=[]

  for label in train_data_labels:
    if label == "POSITIVE":
      train_labels.append(1)
    else:
      train_labels.append(0)
  comment_labels.append(train_labels)

  for label in test_data_labels:
    if label == "POSITIVE":
      test_labels.append(1)
    else:
      test_labels.append(0)
  comment_labels.append(test_labels)
  

  for comment in train_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    train_text.append(lines)
  comment_texts.append(train_text)

  for comment in test_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    test_text.append(lines)
  comment_texts.append(test_text)


  return comment_texts,comment_labels

# preprocess 2

In [0]:
DataTagged = pd.read_csv("/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/corpus/analyzed/comments_tagged_remove_all_punc_keep_question.csv", ";")
train_data, test_data = train_test_split(DataTagged, test_size=0.2, random_state=0)
comment_texts, comment_labels = text_preprocessing(train_data,test_data)

In [6]:
# prepare tokenizer
docs = comment_texts[0]+comment_texts[1]
# print(comment_texts[0])
# print(comment_texts[1])
# print(docs)
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
print(vocab_size)

18416


In [7]:
print(t.word_index.items())

dict_items([('මේ', 1), ('අපේ', 2), ('නම්', 3), ('නැහැ', 4), ('වගේ', 5), ('ඒ', 6), ('ඔබ', 7), ('අපි', 8), ('ගැන', 9), ('එක', 10), ('කරන්න', 11), ('රටේ', 12), ('හරි', 13), ('තමයි', 14), ('මම', 15), ('අය', 16), ('කියලා', 17), ('කරන', 18), ('ඇති', 19), ('දැන්', 20), ('ඔය', 21), ('හොඳ', 22), ('ලංකාවේ', 23), ('ඉන්න', 24), ('මිනිස්සු', 25), ('රට', 26), ('නැති', 27), ('නිසා', 28), ('වෙන්න', 29), ('අනේ', 30), ('වැඩ', 31), ('වැඩක්', 32), ('ඔබට', 33), ('මේක', 34), ('වේවා', 35), ('දෙයක්', 36), ('තියෙන්නේ', 37), ('කියන්නේ', 38), ('ගන්න', 39), ('වලට', 40), ('මගේ', 41), ('එපා', 42), ('අද', 43), ('කරලා', 44), ('අපිට', 45), ('අයට', 46), ('මට', 47), ('රටට', 48), ('දේවල්', 49), ('කියල', 50), ('ලැබේවා', 51), ('සහ', 52), ('මේවා', 53), ('පුළුවන්', 54), ('වෙන', 55), ('වෙනවා', 56), ('කරනවා', 57), ('වෙලා', 58), ('හොද', 59), ('?', 60), ('සුබ', 61), ('යන්න', 62), ('හරිම', 63), ('ගොඩක්', 64), ('යන', 65), ('දෙන්න', 66), ('කළ', 67), ('නෑ', 68), ('කියන', 69), ('වල', 70), ('බව', 71), ('වඩා', 72), ('තව', 73), ('වෙන්නේ

In [8]:
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[132, 104, 444, 18, 206, 1, 5, 4428, 11, 4429, 1532, 374, 76, 159, 32, 6, 9, 422, 16, 2087, 154, 9, 3204], [203, 25], [317, 179, 4430, 302, 7095, 180, 2511, 7096, 6, 4431, 681, 7097, 722, 47, 7098, 7099, 2512, 4432, 1533, 886, 771, 7100, 7101, 1336, 7102, 1337, 7103, 7104, 33, 149, 126], [7105, 267, 2513, 76, 7106, 209, 14, 74, 23, 64, 1338, 1339, 52, 2513, 772, 4433, 246, 56, 93, 2513, 602, 7107, 9, 375, 723, 568, 94], [7108, 2514, 376, 277, 23, 141, 3205, 7109, 17, 287, 7110, 832, 24, 107, 16, 18, 31, 569, 4434], [347, 4435, 59, 31], [1, 7111, 3, 23, 29, 145, 3206, 278, 773, 38, 7112, 1534, 2515, 7113, 318, 1535, 4436, 4], [4437, 682, 196, 52, 3207, 3208, 887, 3209, 1536, 3210, 3211, 3212, 887, 2088, 3213, 3214], [1770, 1771, 603, 66, 570], [4438, 4439, 3, 310, 4440, 5, 14, 4441], [2516, 3215, 62, 7114, 258, 84, 2089, 13, 7115], [1082, 7116, 2517, 333, 7117, 99, 36, 2518, 16, 173], [85, 16, 724, 4442, 7118, 2519, 7119, 279, 2520, 7120, 33, 2, 2090], [1340, 398, 4443, 23, 982, 7121, 

In [9]:
print(docs[0])

['මිනිසුන්', 'තමන්ගේ', 'රැකියා', 'කරන', 'ගමන්', 'මේ', 'වගේ', 'කටයුතුත්', 'කරන්න', 'වෙලාව', 'හදාගෙන', 'කළේ', 'ලොකු', 'පින්', 'වැඩක්', 'ඒ', 'ගැන', 'නොදන්නා', 'අය', 'ලැබුණු', 'මුදල්', 'ගැන', 'අහනවා']


In [10]:
print(encoded_docs[0][1])

104


In [11]:
max_length = 155
padded_docs = pad_sequences(encoded_docs, maxlen=max_length)
print(padded_docs)

[[   0    0    0 ...  154    9 3204]
 [   0    0    0 ...    0  203   25]
 [   0    0    0 ...   33  149  126]
 ...
 [   0    0    0 ...  297 6381 3160]
 [   0    0    0 ...   51    7  242]
 [   0    0    0 ... 1131 6688   39]]


In [12]:
keyed_fasttext_path = '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/word_embedding/fasttext/fasttext_keyed_vectors/keyed.kv'
word_vectors = KeyedVectors.load(keyed_fasttext_path, mmap='r')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
print(len(word_vectors.vocab))
# for word in word_vectors.vocab.items():
#   print(word)


2137729


In [0]:
embeddings_index = dict()
for word ,vocab_obj in word_vectors.vocab.items():
  embeddings_index[word]=word_vectors[word]

In [0]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector


In [16]:
print(embedding_matrix.shape)

(18416, 300)


In [0]:
embedding_matrix_path = '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN/fasttetx_embedding_matrix'

In [0]:

# pickle.dump(embedding_matrix, open(embedding_matrix_path, 'wb'))

In [0]:
# f = open(embedding_matrix_path, 'rb')
# embedding_matrix= np.array(pickle.load(f))

In [0]:
batch_size = 32
nb_filter = 200
filter_length = 4
hidden_dims = nb_filter * 2
nb_epoch = 30
RNN = LSTM
rnn_output_size = 300
folds = 10
vocab_size=embedding_matrix.shape[0]

In [0]:
labels = comment_labels[0]+comment_labels[1]
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=0)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [22]:
print(X_test)

[[    0     0     0 ...     0     0 17044]
 [    0     0     0 ...   238  6121  6122]
 [    0     0     0 ...     0   345  6601]
 ...
 [    0     0     0 ...    33    61   206]
 [    0     0     0 ...   229   128   864]
 [    0     0     0 ...     2   553  1162]]


In [23]:
print(padded_docs)

[[   0    0    0 ...  154    9 3204]
 [   0    0    0 ...    0  203   25]
 [   0    0    0 ...   33  149  126]
 ...
 [   0    0    0 ...  297 6381 3160]
 [   0    0    0 ...   51    7  242]
 [   0    0    0 ... 1131 6688   39]]


In [24]:
maxlen = 155
max_features = vocab_size
embedding_dims = 300

print('Train...')
accs = []
first_run = True

Train...


# Build Model

In [0]:
def build_model():
    main_input = Input(shape=(maxlen, ), dtype='int32', name='main_input')
    embedding  = Embedding(max_features, embedding_dims,
                  weights=[embedding_matrix], input_length=maxlen,
                  name='embedding' ,trainable=True)(main_input)

    embedding = Dropout(0.50)(embedding)

    conv4 = Convolution1D(nb_filter=nb_filter,
                          filter_length=4,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv4')(embedding)
    maxConv4 = MaxPooling1D(pool_length=2,
                             name='maxConv4')(conv4)

    conv5 = Convolution1D(nb_filter=nb_filter,
                          filter_length=5,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv5')(embedding)
    maxConv5 = MaxPooling1D(pool_length=2,
                            name='maxConv5')(conv5)

    # x = Add([maxConv4, maxConv5])
    # x = keras.layers.Add()([maxConv4, maxConv5])
    x = keras.layers.Concatenate(axis=1)([maxConv4, maxConv5])

    x = Dropout(0.15)(x)

    x = RNN(rnn_output_size)(x)

    x = Dense(hidden_dims, activation='relu', init='he_normal',
              W_constraint = maxnorm(3), b_constraint=maxnorm(3),
              name='mlp')(x)

    x = Dropout(0.10, name='drop')(x)

    output = Dense(1, init='he_normal',
                   activation='sigmoid', name='output')(x)

    model = Model(input=main_input, output=output)
    model.compile(loss={'output':'binary_crossentropy'},
                optimizer=Adadelta(lr=0.95, epsilon=1e-06),
                metrics=["accuracy"])
    return model

In [26]:
model = build_model()
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", name="conv4", filters=200, kernel_size=4, strides=1, padding="valid")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(name="maxConv4", pool_size=2)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", name="conv5", filters=200, kernel_size=5, strides=1, padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(name="maxConv5", pool_size=2)`


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 155)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 155, 300)     5524800     main_input[0][0]                 
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 155, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
conv4 (Conv1D)                  (None, 152, 200)     240200      dropout_1[0][0]                  
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, activation="relu", name="mlp", kernel_initializer="he_normal", kernel_constraint=<keras.con..., bias_constraint=<keras.con...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", name="output", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("ma..., outputs=Tensor("ou...)`


# Main

In [27]:
model = build_model()
if first_run:
    first_run = False
    print(model.summary())



best_val_acc = 0
best_test_acc = 0
for j in range(nb_epoch):
    a = time.time()
    his = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    validation_data=[X_test, y_test],
                    shuffle=True,
                    nb_epoch=1, verbose=1)
    print('Epoch %d/%d\t%s' % (j + 1, nb_epoch, str(his.history)))
    if his.history['val_accuracy'][0] >= best_val_acc:
        score, acc = model.evaluate(X_test, y_test,
                                    batch_size=batch_size,
                                    verbose=2)
        best_val_acc = his.history['val_accuracy'][0]
        best_test_acc = acc
        print('Got best epoch  best val acc is %f test acc is %f' %
              (best_val_acc, best_test_acc))
        if len(accs) > 0:
            print('Current avg test acc:', str(np.mean(accs)))
    b = time.time()
    cost = b - a
    left = (nb_epoch - j - 1)
    print('One round cost %ds, %d round %ds %dmin left' % (cost, left,
                                                           cost * left,
                                                           cost * left / 60.0))
accs.append(best_test_acc)
print('Avg test acc:', str(np.mean(accs)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", name="conv4", filters=200, kernel_size=4, strides=1, padding="valid")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(name="maxConv4", pool_size=2)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", name="conv5", filters=200, kernel_size=5, strides=1, padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(name="maxConv5", pool_size=2)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, activation="relu", name="mlp", kernel_in

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 155)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 155, 300)     5524800     main_input[0][0]                 
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 155, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
conv4 (Conv1D)                  (None, 152, 200)     240200      dropout_3[0][0]                  
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4008 samples, validate on 1002 samples
Epoch 1/1
4008/4008 [==============================] - 40s 10ms/step - loss: 0.5159 - accuracy: 0.7488 - val_loss: 0.4343 - val_accuracy: 0.8134
Epoch 1/30	{'val_loss': [0.43425858211136625], 'val_accuracy': [0.8133732676506042], 'loss': [0.515936040711736], 'accuracy': [0.7487525]}
Got best epoch  best val acc is 0.813373 test acc is 0.813373
One round cost 42s, 29 round 1227s 20min left
Train on 4008 samples, validate on 1002 samples
Epoch 1/1
4008/4008 [==============================] - 34s 9ms/step - loss: 0.3914 - accuracy: 0.8244 - val_loss: 0.4709 - val_accuracy: 0.7395
Epoch 2/30	{'val_loss': [0.4709064340639019], 'val_accuracy': [0.7395209670066833], 'loss': [0.391357401292957], 'accuracy': [0.8243513]}
One round cost 34s, 28 round 957s 15min left
Train on 4008 samples, validate on 1002 samples
Epoch 1/1
4008/4008 [==============================] - 34s 8ms/step - loss: 0.3474 - accuracy: 0.8508 - val_loss: 0.4596 - val_accuracy: 